In [2]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from pyclim_noresm.general_util_funcs import global_avg
from workflow.scripts.utils import calc_error
import matplotlib
import numpy as np

In [3]:
paths = sorted(snakemake.input.paths)
vname = snakemake.wildcards.vName
params=snakemake.params
time_slice = params.get('time_slice', slice(3,-1))

In [6]:
erfs = {}
erfs['mean'] = {}
erfs['std'] = {}

for path in paths:
    ds = xr.open_dataset(path)
    ds = ds.rename({'year':'time'})    
    source = ds.parent_source_id
    ts = global_avg(ds[vname])
    ts = ts.mean(dim='time')
    std = calc_error(ds[vname], kind='SEM',time_dim='time')
    erfs['mean'][source] = ts.values
    erfs['std'][source] = std.values
    

In [7]:
df = pd.DataFrame(erfs)
nmodels = len(df['mean'])
multimodel_mean= df['mean'].mean()
error = df['mean'].std()/np.sqrt(nmodels)
df.loc['Multi-model','mean'] = multimodel_mean 
df.loc['Multi-model', 'std'] =error
# df = df.transpose()
vmax=round(abs(df.loc[:,'mean']).max()+0.1,1)
df = df.astype(float)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.axvline(color='darkgrey')
df['mean'].plot.barh(ax=ax, legend=False, xerr=df['std'])
ax.set_xlim(-vmax,vmax)
ax.set_xlabel('$W/m^2$')
plt.savefig(snakemake.output.outpath, bbox_inches='tight', facecolor='white')